In [10]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 300)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.model_selection import train_test_split

In [11]:
df = pd.read_csv('./Data/Ames_HousePrice.csv')
df.shape

(2580, 82)

In [12]:
df['MSSubClass'] = df['MSSubClass'].apply(str)
df['YrSold'] = df['YrSold'].astype(str)
df['MoSold'] = df['MoSold'].astype(str)

In [13]:
c = df.select_dtypes('object')
c.head(3)

,MSSubClass,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
0,30,RL,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,Gable,CompShg,Wd Sdng,Wd Sdng,None,TA,TA,CBlock,TA,TA,No,Rec,Unf,GasA,TA,Y,SBrkr,TA,Typ,Gd,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,3,2010,WD,Normal
1,120,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,Gable,CompShg,HdBoard,HdBoard,BrkFace,Gd,TA,CBlock,Gd,TA,Mn,GLQ,ALQ,GasA,TA,Y,SBrkr,Gd,Typ,NaN,Attchd,Fin,TA,TA,Y,NaN,NaN,NaN,2,2009,WD,Normal
2,30,C (all),Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,None,Gd,TA,BrkTil,TA,TA,No,ALQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,NaN,Detchd,Unf,TA,Po,N,NaN,NaN,NaN,11,2007,WD,Normal


In [14]:
c = c.fillna("None")
c.isnull().sum().sum()

0

In [15]:
c.shape

(2580, 46)

In [16]:
c = pd.get_dummies(c, drop_first=True)
c.shape

(2580, 262)

In [21]:
train = df[df['YrSold']!='2010']
train.shape

(2265, 82)

In [25]:
c_train = train.select_dtypes('object')
c_train.shape

(2265, 46)

In [29]:
c_train = c_train.fillna('None')
c_train.isnull().sum().sum()

0

In [30]:
c_train = pd.get_dummies(c_train, drop_first=True)
c_train.shape

(2265, 259)

In [31]:
#difference between original c and c_train without 2010 obs:
c.columns.difference(c_train.columns, sort=False)

Index(['ExterCond_Po', 'KitchenQual_Po', 'YrSold_2010'], dtype='object')

In [34]:
n_train = train.select_dtypes('number')
n_train.head(3)

,Unnamed: 0,PID,GrLivArea,SalePrice,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal
1,2,905476230,1049,139500,42.0,4235,5,5,1984,1984,149.0,552.0,393.0,104.0,1049.0,1049,0,0,1.0,0.0,2,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,0
2,3,911128020,1001,124900,60.0,6060,5,9,1930,2007,0.0,737.0,0.0,100.0,837.0,1001,0,0,0.0,0.0,1,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,0
3,4,535377150,1039,114000,80.0,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,405.0,717,322,0,0.0,0.0,1,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,0


In [36]:
nan_cols = [i for i in n_train.columns if n_train[i].isnull().any()]
nan_cols

['LotFrontage',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea']

In [37]:
n_train[nan_cols].isna().sum()

LotFrontage     415
MasVnrArea       13
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt     108
GarageCars        1
GarageArea        1
dtype: int64

In [38]:
n_train[nan_cols].mean()

LotFrontage       68.555135
MasVnrArea       101.388544
BsmtFinSF1       443.330830
BsmtFinSF2        52.591873
BsmtUnfSF        539.879417
TotalBsmtSF     1035.802120
BsmtFullBath       0.429518
BsmtHalfBath       0.064074
GarageYrBlt     1976.975429
GarageCars         1.758834
GarageArea       469.230565
dtype: float64

In [39]:
for cols in nan_cols:
    mean_value = n_train[cols].mean()
    n_train[cols].fillna(value=mean_value, inplace=True)
    print (cols, mean_value)

LotFrontage 68.55513513513513
MasVnrArea 101.3885435168739
BsmtFinSF1 443.33083038869256
BsmtFinSF2 52.591872791519435
BsmtUnfSF 539.8794169611307
TotalBsmtSF 1035.8021201413428
BsmtFullBath 0.4295183384887318
BsmtHalfBath 0.06407423773751657
GarageYrBlt 1976.9754288363467
GarageCars 1.7588339222614842
GarageArea 469.23056537102474


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [40]:
n_train.shape

(2265, 36)

In [42]:
n_train.isnull().sum().sum()

0

In [44]:
full_train = pd.concat([c_train, n_train], axis=1)
full_train.shape

(2265, 295)

In [45]:
full_train.to_csv('TrainByYrData.csv')

In [32]:
test = df[df['YrSold']=='2010']
test.shape

(315, 82)

In [46]:
c_test = test.select_dtypes('object')
c_test.shape

(315, 46)

In [47]:
c_test = c_test.fillna('None')
c_test.isnull().sum().sum()

0

In [48]:
c_test = pd.get_dummies(c_test, drop_first=True)
c_test.shape

(315, 199)